In [1]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR, MultiStepLR
import numpy as np
# import matplotlib.pyplot as plt
from math import *
import time
torch.cuda.set_device(2)
torch.set_default_tensor_type('torch.DoubleTensor')

In [2]:
# activation function
def activation(x):
    return x * torch.sigmoid(x) 

In [3]:
# exact solution
def u_ex(x):     
    temp = 1.0
    for i in range(space_dimension):
        temp = temp * torch.sin(pi*x[:, i])
    u_temp = x[:, -1] * temp
    return u_temp.reshape([x.size()[0], 1])

In [4]:
def f(x):
    temp = 1.0
    for i in range(space_dimension):
        temp = temp * torch.sin(pi*x[:, i])
    f_temp = (1.0 + space_dimension * x[:, -1] * pi**2) * temp
    return f_temp.reshape([x.size()[0], 1])

In [5]:
# build ResNet with three blocks
class Net(nn.Module):
    def __init__(self,input_size,width,output_size):
        super(Net,self).__init__()
        self.layer_in = nn.Linear(input_size,width)
        self.layer_1 = nn.Linear(width,width)
        self.layer_2 = nn.Linear(width,width)
        self.layer_3 = nn.Linear(width,width)
        self.layer_4 = nn.Linear(width,width)
        self.layer_5 = nn.Linear(width,width)
        self.layer_6 = nn.Linear(width,width)
        self.layer_out = nn.Linear(width,output_size)
    def forward(self,x):
        output = self.layer_in(x) 
        output = output + activation(self.layer_2(activation(self.layer_1(output)))) # residual block 1
        output = output + activation(self.layer_4(activation(self.layer_3(output)))) # residual block 2
        output = output + activation(self.layer_6(activation(self.layer_5(output)))) # residual block 3
        output = self.layer_out(output)
        return output

In [6]:
time_dimension = 1
space_dimension = 3
d = space_dimension + time_dimension # dimension of input include time and space variables
input_size = d 
width_1 = 8
width_2 = 8
output_size_1 = 1 
output_size_2 = d 
data_size = 2000

In [7]:
CUDA = torch.cuda.is_available()
# print('CUDA is: ', CUDA)
if CUDA:
    net_1 = Net(input_size, width_1, output_size_1).cuda() # network for u on gpu
    net_2 = Net(input_size, width_2, output_size_2).cuda() # network for grad u and u_t on gpu
else:
    net_1 = Net(input_size, width_1, output_size_1) # network for u on cpu
    net_2 = Net(input_size, width_2, output_size_2) # network for grad u and u_t on cpu

In [8]:
def model_u(x):
    x_temp = (x[:,0:d-1]).cuda()
    D_x_0 = torch.prod(x_temp, axis = 1).reshape([x.size()[0], 1]) 
    D_x_1 = torch.prod(1.0 - x_temp, axis = 1).reshape([x.size()[0], 1]) 
    model_u_temp = D_x_0 * D_x_1 * (x[:, -1]).reshape([x.size()[0], 1]) * net_1(x)
    return model_u_temp.reshape([x.size()[0], 1])

In [9]:
def generate_sample(data_size_temp):
    sample_temp = torch.rand(data_size_temp, d)
    return sample_temp.cuda()

In [10]:
def relative_l2_error():
    data_size_temp = 500
    x = generate_sample(data_size_temp).cuda() 
    predict = model_u(x)
    exact = u_ex(x)
    value = torch.sqrt(torch.sum((predict - exact)**2))/torch.sqrt(torch.sum((exact)**2))
    return value

In [11]:
# Xavier normal initialization for weights:
#             mean = 0 std = gain * sqrt(2 / fan_in + fan_out)
# zero initialization for biases
def initialize_weights(self):
    for m in self.modules():
        if isinstance(m,nn.Linear):
            nn.init.xavier_normal_(m.weight.data)
            if m.bias is not None:
                m.bias.data.zero_()

In [12]:
initialize_weights(net_1)
initialize_weights(net_2)

In [13]:
def loss_function(x):
#     x = generate_sample(data_size).cuda()
#     x.requires_grad = True
    u_hat = model_u(x)
    p = net_2(x)
    p_x = (p[:, 0:d-1]).reshape([x.size()[0], d - 1])
    p_t = (p[:, -1]).reshape([x.size()[0], 1])
    
    laplace_u_hat = torch.zeros([x.size()[0], 1]).cuda()
    for index in range(space_dimension):
        p_temp = p[:, index].reshape([x.size()[0], 1])
        temp = torch.autograd.grad(outputs = p_temp, inputs = x, grad_outputs = torch.ones(p_temp.shape).cuda(), create_graph = True, allow_unused = True)[0]
        laplace_u_hat = temp[:, index].reshape([x.size()[0], 1]) + laplace_u_hat
    part_1 = torch.sum((p_t -laplace_u_hat - f(x))**2) / len(x)
    
    grad_u_hat = torch.autograd.grad(outputs = u_hat, inputs = x, grad_outputs = torch.ones(u_hat.shape).cuda(), create_graph = True)
    part_2 = torch.sum(((grad_u_hat[0][:, 0:d-1]).reshape([x.size()[0], d - 1]) - p_x)**2) / len(x)
    
    u_hat_t = grad_u_hat[0][:, -1].reshape([x.size()[0], 1])
    part_3 = torch.sum((u_hat_t - p[:, -1].reshape([x.size()[0], 1]))**2) / len(x)
    return part_1 + part_2 + part_3

In [14]:
optimizer = optim.Adam([
                {'params': net_1.parameters()},
                {'params': net_2.parameters()},
            ])

In [15]:
epoch = 50000
loss_record = np.zeros(epoch)
error_record = np.zeros(epoch)
time_start = time.time()
for i in range(epoch):
    optimizer.zero_grad()
    x = generate_sample(data_size).cuda()
    x.requires_grad = True
    loss = loss_function(x)
    loss_record[i] = float(loss)
    error = relative_l2_error()
    error_record[i] = float(error)
    np.save("MIM_loss_parabolic_3d.npy", loss_record)
    np.save("MIM_error_parabolic_3d.npy", error_record)
    if i % 50 == 0:
        print("current epoch is: ", i)
        print("current loss is: ", loss.detach())
        print("current error is: ", error.detach())
    loss.backward()
    optimizer.step() 
time_end = time.time()
print('total time is: ', time_end-time_start, 'seconds')

current epoch is:  0
current loss is:  tensor(32.5246, device='cuda:3')
current error is:  tensor(1.0053, device='cuda:3')
current epoch is:  50
current loss is:  tensor(24.9978, device='cuda:3')
current error is:  tensor(0.9636, device='cuda:3')
current epoch is:  100
current loss is:  tensor(22.1987, device='cuda:3')
current error is:  tensor(0.8288, device='cuda:3')
current epoch is:  150
current loss is:  tensor(21.7184, device='cuda:3')
current error is:  tensor(0.5426, device='cuda:3')
current epoch is:  200
current loss is:  tensor(20.1323, device='cuda:3')
current error is:  tensor(0.5201, device='cuda:3')
current epoch is:  250
current loss is:  tensor(17.4170, device='cuda:3')
current error is:  tensor(0.4464, device='cuda:3')
current epoch is:  300
current loss is:  tensor(15.7930, device='cuda:3')
current error is:  tensor(0.3999, device='cuda:3')
current epoch is:  350
current loss is:  tensor(12.4094, device='cuda:3')
current error is:  tensor(0.3685, device='cuda:3')
cur

current epoch is:  3300
current loss is:  tensor(0.2588, device='cuda:3')
current error is:  tensor(0.0289, device='cuda:3')
current epoch is:  3350
current loss is:  tensor(0.2630, device='cuda:3')
current error is:  tensor(0.0386, device='cuda:3')
current epoch is:  3400
current loss is:  tensor(0.2520, device='cuda:3')
current error is:  tensor(0.0349, device='cuda:3')
current epoch is:  3450
current loss is:  tensor(0.2538, device='cuda:3')
current error is:  tensor(0.0278, device='cuda:3')
current epoch is:  3500
current loss is:  tensor(0.2539, device='cuda:3')
current error is:  tensor(0.0308, device='cuda:3')
current epoch is:  3550
current loss is:  tensor(0.2456, device='cuda:3')
current error is:  tensor(0.0289, device='cuda:3')
current epoch is:  3600
current loss is:  tensor(0.2462, device='cuda:3')
current error is:  tensor(0.0323, device='cuda:3')
current epoch is:  3650
current loss is:  tensor(0.2339, device='cuda:3')
current error is:  tensor(0.0320, device='cuda:3')


current epoch is:  6600
current loss is:  tensor(0.1143, device='cuda:3')
current error is:  tensor(0.0221, device='cuda:3')
current epoch is:  6650
current loss is:  tensor(0.1129, device='cuda:3')
current error is:  tensor(0.0238, device='cuda:3')
current epoch is:  6700
current loss is:  tensor(0.1199, device='cuda:3')
current error is:  tensor(0.0246, device='cuda:3')
current epoch is:  6750
current loss is:  tensor(0.1180, device='cuda:3')
current error is:  tensor(0.0233, device='cuda:3')
current epoch is:  6800
current loss is:  tensor(0.1102, device='cuda:3')
current error is:  tensor(0.0272, device='cuda:3')
current epoch is:  6850
current loss is:  tensor(0.1092, device='cuda:3')
current error is:  tensor(0.0248, device='cuda:3')
current epoch is:  6900
current loss is:  tensor(0.1083, device='cuda:3')
current error is:  tensor(0.0235, device='cuda:3')
current epoch is:  6950
current loss is:  tensor(0.1084, device='cuda:3')
current error is:  tensor(0.0268, device='cuda:3')


current epoch is:  9900
current loss is:  tensor(0.0839, device='cuda:3')
current error is:  tensor(0.0222, device='cuda:3')
current epoch is:  9950
current loss is:  tensor(0.0838, device='cuda:3')
current error is:  tensor(0.0182, device='cuda:3')
current epoch is:  10000
current loss is:  tensor(0.0838, device='cuda:3')
current error is:  tensor(0.0198, device='cuda:3')
current epoch is:  10050
current loss is:  tensor(0.0841, device='cuda:3')
current error is:  tensor(0.0212, device='cuda:3')
current epoch is:  10100
current loss is:  tensor(0.0849, device='cuda:3')
current error is:  tensor(0.0210, device='cuda:3')
current epoch is:  10150
current loss is:  tensor(0.0839, device='cuda:3')
current error is:  tensor(0.0184, device='cuda:3')
current epoch is:  10200
current loss is:  tensor(0.0828, device='cuda:3')
current error is:  tensor(0.0200, device='cuda:3')
current epoch is:  10250
current loss is:  tensor(0.0863, device='cuda:3')
current error is:  tensor(0.0212, device='cud

current epoch is:  13200
current loss is:  tensor(0.0733, device='cuda:3')
current error is:  tensor(0.0174, device='cuda:3')
current epoch is:  13250
current loss is:  tensor(0.0680, device='cuda:3')
current error is:  tensor(0.0184, device='cuda:3')
current epoch is:  13300
current loss is:  tensor(0.0789, device='cuda:3')
current error is:  tensor(0.0159, device='cuda:3')
current epoch is:  13350
current loss is:  tensor(0.0721, device='cuda:3')
current error is:  tensor(0.0152, device='cuda:3')
current epoch is:  13400
current loss is:  tensor(0.0699, device='cuda:3')
current error is:  tensor(0.0193, device='cuda:3')
current epoch is:  13450
current loss is:  tensor(0.0733, device='cuda:3')
current error is:  tensor(0.0156, device='cuda:3')
current epoch is:  13500
current loss is:  tensor(0.0676, device='cuda:3')
current error is:  tensor(0.0182, device='cuda:3')
current epoch is:  13550
current loss is:  tensor(0.0725, device='cuda:3')
current error is:  tensor(0.0185, device='c

current epoch is:  16500
current loss is:  tensor(0.0659, device='cuda:3')
current error is:  tensor(0.0164, device='cuda:3')
current epoch is:  16550
current loss is:  tensor(0.0615, device='cuda:3')
current error is:  tensor(0.0131, device='cuda:3')
current epoch is:  16600
current loss is:  tensor(0.0631, device='cuda:3')
current error is:  tensor(0.0163, device='cuda:3')
current epoch is:  16650
current loss is:  tensor(0.0637, device='cuda:3')
current error is:  tensor(0.0161, device='cuda:3')
current epoch is:  16700
current loss is:  tensor(0.0615, device='cuda:3')
current error is:  tensor(0.0133, device='cuda:3')
current epoch is:  16750
current loss is:  tensor(0.0648, device='cuda:3')
current error is:  tensor(0.0132, device='cuda:3')
current epoch is:  16800
current loss is:  tensor(0.0651, device='cuda:3')
current error is:  tensor(0.0140, device='cuda:3')
current epoch is:  16850
current loss is:  tensor(0.0646, device='cuda:3')
current error is:  tensor(0.0143, device='c

current epoch is:  19800
current loss is:  tensor(0.0611, device='cuda:3')
current error is:  tensor(0.0133, device='cuda:3')
current epoch is:  19850
current loss is:  tensor(0.0582, device='cuda:3')
current error is:  tensor(0.0114, device='cuda:3')
current epoch is:  19900
current loss is:  tensor(0.0582, device='cuda:3')
current error is:  tensor(0.0144, device='cuda:3')
current epoch is:  19950
current loss is:  tensor(0.0570, device='cuda:3')
current error is:  tensor(0.0134, device='cuda:3')
total time is:  2076.3964536190033 seconds


In [16]:
np.save("MIM_loss_parabolic_3d.npy", loss_record)
np.save("MIM_error_parabolic_3d.npy", error_record)